In [1]:
import pandas as pd
import datetime
from config import serp_api
from serpapi import GoogleSearch
import pprint

### Aggregating Google Jobs Data Using SerpAPI.com
The data for this project is obtained from SerpAPI.com's Google Jobs API, which is documented here: https://serpapi.com/google-jobs-api.

To facilitate daily searches, I have created functions that allow for the search of either a single page with 10 job listings or up to 60 job listings. These functions take a job title string as an argument and can be easily modified to accommodate different job titles for daily searches.

In [2]:
# This function creates one page of Google job results from the Google Jobs API endpoint for the job_title arguement.
# The 'uule' setting is to capture listings for remote work.

def search_ten_and_create_df(job_title):
    params= {
        "q": f'{job_title}',
        "ibp": "htl;jobs",
        "uule": "w+CAIQICINVW5pdGVkIFN0YXRlcw",
        "hl": "en",
        "gl": "us",
        "ltype": "1",
        "api_key":f'{serp_api}',
        "engine":"google_jobs"
    }

    search = GoogleSearch(params)
    results = search.get_dict()
    jobs_results = results["jobs_results"]

    job_data = []
    for i in range(len(jobs_results)):
        job_data.append({
        'job title': jobs_results[i]['title'],
        'company name': jobs_results[i]['company_name'],
        'job description': jobs_results[i]['description'],
        'via': jobs_results[i]['via'],
        'job_id': jobs_results[i]['job_id'],
        'start':"0"
        })

    ten_jobs_df = pd.DataFrame(job_data)

     # Adding the current date and time to track whent he listing was captured in my data
    current_date_time = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    ten_jobs_df['report_run_datetime'] = current_date_time

    return ten_jobs_df


In [5]:
# Creating a DataFrame using the search_ten function for the job title 'Data Analyst'.
small_result_df = search_ten_and_create_df('data analyst')

https://serpapi.com/search


In [3]:
# This function creates 6 pages of Google job results from the Google Jobs API endpoint for the job_title arguement. 
# The 'uule' setting is to capture remote only listings

def search_sixty_and_create_df(job_title):
    all_jobs_data = []
    start_values = [0, 10, 20, 30, 40, 50]
    
    for start in start_values:
        params= {
            "q": f'{job_title}',
            "ibp": "htl;jobs",
            "uule": "w+CAIQICINVW5pdGVkIFN0YXRlcw",
            "hl": "en",
            "gl": "us",
            "ltype": "1",
            "api_key":f'{serp_api}',
            "engine":"google_jobs",
            "start": f'{start}'
        }

        search = GoogleSearch(params)
        results = search.get_dict()
        jobs_results = results["jobs_results"]
        
        # Using the get method on the dictionary, which returns None if the key is not present in the dictionary.
        # The [{}] syntax creates an empty dictionary within a list
        jobs_data = []
        for i in range(len(jobs_results)):
            jobs_data.append({
            'job title': jobs_results[i].get('title', None),
            'company name': jobs_results[i].get('company_name', None),
            'job description': jobs_results[i].get('description', None),
            'via': jobs_results[i].get('via', None),
            'job_id': jobs_results[i].get('job_id', None),
            'posted': jobs_results[i]['detected_extensions'].get('posted_at', None)
            })

        all_jobs_data.extend(jobs_data)

    # Creating a DataFrame from the list of dictionaries
    jobs_df = pd.DataFrame(all_jobs_data)
    
    # Adding the current date and time to track whent he listing was captured in my data
    current_date_time = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    jobs_df['report_run_datetime'] = current_date_time
    
    return jobs_df


In [4]:
# Creating a 60 listing DataFrame for the job title 'Operations Analyst' using the search_sixty function.
oa_google_jobs_df = search_sixty_and_create_df('operations analyst')

https://serpapi.com/search
https://serpapi.com/search
https://serpapi.com/search
https://serpapi.com/search
https://serpapi.com/search
https://serpapi.com/search


In [5]:
# Creating a 60 listing DataFrame for the job title 'Data Analyst' using the search_sixty function.
da_google_jobs_df = search_sixty_and_create_df('data analyst')

https://serpapi.com/search
https://serpapi.com/search
https://serpapi.com/search
https://serpapi.com/search
https://serpapi.com/search
https://serpapi.com/search


### Capturing Details for the Google Jobs Listings

Using the SerpAPI's Google Jobs listing endpoint, I have developed a function that takes any number of DataFrames as an argument and captures job listing details such as company rating, number of rating reviews, review source, and salary information. This function enhances the available information in the DataFrames to enable better analysis and insights into the job market.

In [6]:
def get_job_list_details(*dataframes):
    final_df = pd.DataFrame()
    
    # Looping through each dataframe passed as argument
    for dataframe in dataframes:
        id_results_list = []
        
        # Looping through each row of the current dataframe
        for index, row in dataframe.iterrows():
            job_id = row['job_id']
            params = {
                "q": f"{job_id}",
                "api_key": f"{serp_api}",
                "engine": "google_jobs_listing"
            }
            search = GoogleSearch(params)
            results = search.get_dict()
            
            # Appending the results of the API call to a list
            id_results_list.append(results)
        
        id_jobs_data = []
        
        # Looping through the list of results and extracting listing information
        for i in range(len(id_results_list)):
            id_jobs_data.append({
            'apply options': id_results_list[i].get('apply_options', [{}])[0].get('link', None),
            'rating': id_results_list[i].get('ratings', [{}])[0].get('rating', None),
            '# of reviews': id_results_list[i].get('ratings', [{}])[0].get('reviews', None),
            'rating source': id_results_list[i].get('ratings', [{}])[0].get('source', None),
            'salary based on': id_results_list[i].get('salaries', [{}])[0].get('based_on', None),
            'salary from': id_results_list[i].get('salaries', [{}])[0].get('salary_from', None),
            'salary to': id_results_list[i].get('salaries', [{}])[0].get('salary_to', None),
            'salary source': id_results_list[i].get('salaries', [{}])[0].get('source', None),
            'job_id': id_results_list[i].get('search_parameters', [{}]).get('q', None)
            })
        
        # Creating a DataFrame from the list of dictionaries
        df = pd.DataFrame(id_jobs_data)
        
        # Adding the information from the original DataFrame to the new DataFrame
        df = pd.concat([dataframe, df], axis=1)
        
        # Concatenating the current DataFrame to the final DataFrame
        final_df = pd.concat([final_df, df], axis=0)
        
    return final_df


In [7]:
# Pulling the job details for all of the listings captured in the dataframes above
all_jobs_3_8_2023_df = get_job_list_details(oa_google_jobs_df, da_google_jobs_df)

https://serpapi.com/search
https://serpapi.com/search
https://serpapi.com/search
https://serpapi.com/search
https://serpapi.com/search
https://serpapi.com/search
https://serpapi.com/search
https://serpapi.com/search
https://serpapi.com/search
https://serpapi.com/search
https://serpapi.com/search
https://serpapi.com/search
https://serpapi.com/search
https://serpapi.com/search
https://serpapi.com/search
https://serpapi.com/search
https://serpapi.com/search
https://serpapi.com/search
https://serpapi.com/search
https://serpapi.com/search
https://serpapi.com/search
https://serpapi.com/search
https://serpapi.com/search
https://serpapi.com/search
https://serpapi.com/search
https://serpapi.com/search
https://serpapi.com/search
https://serpapi.com/search
https://serpapi.com/search
https://serpapi.com/search
https://serpapi.com/search
https://serpapi.com/search
https://serpapi.com/search
https://serpapi.com/search
https://serpapi.com/search
https://serpapi.com/search
https://serpapi.com/search
h

In [ ]:
# Saving the dataframe to a csv for analysis in a seperate notebook
all_jobs_3_8_2023_df.to_csv('google_jobs_3_8_2023.csv', index=False)